# Q4/a

In [178]:
import pandas as pd
import numpy as np
import datetime as dt
from pandas_datareader import data as pdr
import yfinance as yf

yf.pdr_override()

stocks = ['AAPL', 'BP', 'COST', 'GOOG', 'GS', 'IBM', 'XOM']

endDate = "2016-12-31" #no stock data for 2016/12/31, so no need to extend date
startDate = "2016-01-01" #no stock data for 2016/01/01 - 2016/01/03, so no need to extend date

# Import stock data
def getData(stocks, start, end):
    stockData = pdr.get_data_yahoo(stocks, start=start, end=end)
    stockData = stockData['Close']
    returns = stockData.pct_change()
    meanReturns = returns.mean()
    covMatrix = returns.cov()
    return returns, meanReturns, covMatrix


returns, meanReturns, covMatrix = getData(stocks, start=startDate, end=endDate)
returns = returns.dropna()

weights = [0.15,0.15,0.15,0.20,0.05,0.20,0.10]

confidence_lvl=5

returns['portfolio'] = returns.dot(weights)



[*********************100%***********************]  7 of 7 completed


In [179]:

hist_VAR_1 = -np.percentile(returns['portfolio'], confidence_lvl)*np.sqrt(1)
hist_CVAR_1 = -returns['portfolio'][returns['portfolio'] <= np.percentile(returns['portfolio'], confidence_lvl)].mean()*np.sqrt(1)

print("1-day Hostorical VAR95% is:")
print(hist_VAR_1)
print(" ")
print("1-day Hostorical CVAR95% is:")
print(hist_CVAR_1)




1-day Hostorical VAR95% is:
0.01416876747455724
 
1-day Hostorical CVAR95% is:
0.021256080522287897


In [180]:
hist_VAR_252 = -np.percentile(returns['portfolio'], confidence_lvl)*np.sqrt(252)
hist_CVAR_252 = -returns['portfolio'][returns['portfolio'] <= np.percentile(returns['portfolio'], confidence_lvl)].mean()*np.sqrt(252)

print("252-day Hostorical VAR95% is:")
print(hist_VAR_252)
print(" ")
print("252-day Hostorical CVAR95% is:")
print(hist_CVAR_252)

252-day Hostorical VAR95% is:
0.22492221073187488
 
252-day Hostorical CVAR95% is:
0.3374298174596263


# Q4/b

In [181]:
from scipy.stats import norm

# Portfolio Performance
def portfolioPerformance(weights, meanReturns, covMatrix, Time):
    returns = np.sum(meanReturns*weights)*Time
    std = np.sqrt( np.dot(weights.T, np.dot(covMatrix, weights)) ) * np.sqrt(Time)
    return returns, std



In [182]:
Time_1=1
pRet_1, pStd_1 = portfolioPerformance(np.array(weights), meanReturns, covMatrix, Time_1)
para_VAR_1 = -((-norm.ppf(1-confidence_lvl/100))*pStd_1 + pRet_1)
para_CVAR_1 = (confidence_lvl/100)**-1 * norm.pdf(norm.ppf(confidence_lvl/100))*pStd_1 - pRet_1


print("1-day parametric VAR95% is:")
print(para_VAR_1)
print(" ")
print("1-day parametric CVAR95% is:")
print(para_CVAR_1)






1-day parametric VAR95% is:
0.014410025028747123
 
1-day parametric CVAR95% is:
0.01820605798416901


In [183]:
Time_252=252
pRet_252, pStd_252 = portfolioPerformance(np.array(weights), meanReturns, covMatrix, Time_252)
para_VAR_252 = -((-norm.ppf(1-confidence_lvl/100))*pStd_252 + pRet_252)
para_CVAR_252 = (confidence_lvl/100)**-1 * norm.pdf(norm.ppf(confidence_lvl/100))*pStd_252 - pRet_252

print("252-day parametric VAR95% is:")
print(para_VAR_252)
print(" ")
print("252-day parametric CVAR95% is:")
print(para_CVAR_252)

252-day parametric VAR95% is:
0.10298849653931971
 
252-day parametric CVAR95% is:
0.16324865155123086
